<a href="https://colab.research.google.com/github/R-Mosolov/sociology-scientometric-analysis/blob/main/analyze_articles_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
ABOUT THE MODULE:
This module contains logic to analyze about 50000 article texts
from "CyberLeninka" as a Russian scientific database
'''

'''
NOTATIONS:
– df – a dataframe/dataset, or an array of initial data.
'''

'\nNOTATIONS:\n– df – a dataframe/dataset, or an array of initial data.\n'

In [2]:
from bs4 import BeautifulSoup
import urllib.request
import requests
import csv
import pandas as pd
import random
import time
import os
import numpy as np
import statistics

# !pip uninstall pandas_profiling
# !pip install pandas_profiling
from pandas_profiling import ProfileReport

# !pip uninstall nltk
# !pip install nltk
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [3]:
'''
Initialize the dataframe
'''
df = pd.read_csv(
  '/content/drive/MyDrive/Science/Datasets/cyberleninka-sociology-articles/cyberleninka-sociology-articles'
  + '__2-0_articles-content/integrated-dataframes/integrated-dataframe__actuality-09-01-2021-2.csv'
)

In [4]:
'''
Open the dataframe
'''
df

,article_link,article_views,article_downloads,journal_title,journal_link,key_words,abstract,article_text
0,/article/n/sravnitelnaya-harakteristika-sotsia...,747,80,Ученые записки университета им. П.Ф. Лесгафта,/journal/n/uchenye-zapiski-universiteta-im-p-f...,EMPTY,EMPTY,['\ufeffСРАВНИТЕЛЬНАЯ ХАРАКТЕРИСТИКА СОЦИАЛЬНО...
1,/article/n/sotsialno-pedagogicheskieiusloviya-...,226,44,"Мир науки, культуры, образования",/journal/n/mir-nauki-kultury-obrazovaniya,['социально-нравственная направленность личнос...,В статье раскрываются социально-педагогические...,['В статье раскрываются социально-педагогическ...
2,/article/n/vliyanie-kachestva-zhizni-naseleniy...,1045,70,Власть,/journal/n/vlast,EMPTY,Главная особенность муниципального уровня упра...,['Главная особенность муниципального уровня уп...
3,/article/n/informatsionnye-tehnologii-v-mestno...,1408,97,Известия Южного федерального университета. Тех...,/journal/n/izvestiya-yuzhnogo-federalnogo-univ...,EMPTY,EMPTY,"['\ufeffгорода. В-третьих, как изменилось благ..."
4,/article/n/tsennostnye-orientatsii-v-semeynoy-...,5254,588,Теория и практика общественного развития,/journal/n/teoriya-i-praktika-obschestvennogo-...,"['семья', 'брак', 'ценности', 'family', 'marri...",Статья раскрывает основные подходы к определен...,['Статья раскрывает основные подходы к определ...
...,...,...,...,...,...,...,...,...
46952,/article/n/informatsionno-psihologicheskaya-be...,17,4,Педагогический журнал Башкортостана,/journal/n/pedagogicheskiy-zhurnal-bashkortostana,"['информационно-психологическое влияние', 'сре...",В статье рассматривается сущность и характер и...,['В статье рассматривается сущность и характер...
46953,/article/n/zaschita-ot-novyh-sotsialnyh-riskov...,10,2,"Российское право: образование, практика, наука",/journal/n/rossiyskoe-pravo-obrazovanie-prakti...,"['причины и факторы социальных рисков', 'новые...",Повышенное внимание к рискам во всех сферах об...,['Повышенное внимание к рискам во всех сферах ...
46954,/article/n/teoreticheskie-i-pravovye-aspekty-p...,5,1,Вестник Московского университета МВД России,/journal/n/vestnik-moskovskogo-universiteta-mv...,"['социализация', 'ребенок', 'родитель', 'особе...",Процесс заботливого отношения общества к лицам...,['Процесс заботливого отношения общества к лиц...
46955,/article/n/kulturnaya-identichnost-kak-faktor-...,4,0,Психология и педагогика служебной деятельности,/journal/n/psihologiya-i-pedagogika-sluzhebnoy...,"['безопасность', 'суверенитет', 'образование',...",В статье рассматриваются условия обеспечения б...,['В статье рассматриваются условия обеспечения...


In [5]:
'''
Create decorder for texts from string to array type
'''
firstArticleText = df['article_text'][0]

def transformFromStrToArr(textAsStr):
  textWithoutOuterWrapper = textAsStr.replace(']', '').replace('[', '')
  textAsArr = textWithoutOuterWrapper.replace("'", '').split(', ')
  return textAsArr

transformFromStrToArr(firstArticleText)

['\\ufeffСРАВНИТЕЛЬНАЯ ХАРАКТЕРИСТИКА СОЦИАЛЬНОЙ ДИФФЕРЕНЦИАЦИИ РОССИЙСКОГО И АМЕРИКАНСКОГО ОБЩЕСТВ',
 'С. И. Росенко',
 'Последние десятилетия ознаменовались существенными изменениями качественного и количественного состояния социальной дифференциации постиндустриального общества. Динамика социальных изменений характеризуется такими тенденциями как появление новых групп и слоев',
 'рост социальной мобильности',
 'перемещение занятого населения в сферу услуг',
 'трансформация профессиональной структуры',
 'дальнейшее имущественное расслоение населения и др. Наряду с общей направленностью эволюции современного социума',
 'существуют особенности развития отдельных государств',
 'определяемые состоянием их экономики',
 'уровнем и характером производства',
 'содержанием политических отношений и т.д. Россия и Соединенные Штаты Америки занимают одно из центральных мест в мировой экономике и политике. В этой связи сравнительная характеристика эволюции социальной дифференциации в этих странах 

In [6]:
'''
Open first article text
'''
firstArticleText

"['\\ufeffСРАВНИТЕЛЬНАЯ ХАРАКТЕРИСТИКА СОЦИАЛЬНОЙ ДИФФЕРЕНЦИАЦИИ РОССИЙСКОГО И АМЕРИКАНСКОГО ОБЩЕСТВ', 'С. И. Росенко', 'Последние десятилетия ознаменовались существенными изменениями качественного и количественного состояния социальной дифференциации постиндустриального общества. Динамика социальных изменений характеризуется такими тенденциями как появление новых групп и слоев, рост социальной мобильности, перемещение занятого населения в сферу услуг, трансформация профессиональной структуры, дальнейшее имущественное расслоение населения и др. Наряду с общей направленностью эволюции современного социума, существуют особенности развития отдельных государств, определяемые состоянием их экономики, уровнем и характером производства, содержанием политических отношений и т.д. Россия и Соединенные Штаты Америки занимают одно из центральных мест в мировой экономике и политике. В этой связи сравнительная характеристика эволюции социальной дифференциации в этих странах составляет важную предпос

In [7]:
'''
Restore text structure of an one random text
'''
textAsArr = transformFromStrToArr(firstArticleText)

SPACE = ' '
HYPHEN_1 = '-'
HYPHEN_2 = '–'
HYPHEN_3 = '—'

'''
Additional functions
'''
def isLastTextChunk(idx, textAsArr):
  idx == len(textAsArr) - 1

def isEndsOnHyphen(textAsStr):
  lastChunkSymbol = textAsArr[len(textAsArr) - 1]
  if (
    lastChunkSymbol == HYPHEN_1
    or lastChunkSymbol == HYPHEN_2
    or lastChunkSymbol == HYPHEN_3
  ):
    return true

'''
Main function
'''
def restoreOneArticleText(textAsArr):
  restoredText = ''

  for idx, textChunk in enumerate(textAsArr):
    
    # Pass last text chunk because it doesn't require handling for chunk end
    if isLastTextChunk(idx, textChunk):
      restoredText += textChunk
    
    if isEndsOnHyphen(textChunk):
      textChunkWithoutHyphen = textChunk[:-1]
      restoredText += textChunkWithoutHyphen

    # Handle other universal cases
    else:
      restoredText += textChunk + SPACE

  return restoredText

restoreOneArticleText(textAsArr)

'\\ufeffСРАВНИТЕЛЬНАЯ ХАРАКТЕРИСТИКА СОЦИАЛЬНОЙ ДИФФЕРЕНЦИАЦИИ РОССИЙСКОГО И АМЕРИКАНСКОГО ОБЩЕСТВ С. И. Росенко Последние десятилетия ознаменовались существенными изменениями качественного и количественного состояния социальной дифференциации постиндустриального общества. Динамика социальных изменений характеризуется такими тенденциями как появление новых групп и слоев рост социальной мобильности перемещение занятого населения в сферу услуг трансформация профессиональной структуры дальнейшее имущественное расслоение населения и др. Наряду с общей направленностью эволюции современного социума существуют особенности развития отдельных государств определяемые состоянием их экономики уровнем и характером производства содержанием политических отношений и т.д. Россия и Соединенные Штаты Америки занимают одно из центральных мест в мировой экономике и политике. В этой связи сравнительная характеристика эволюции социальной дифференциации в этих странах составляет важную предпосылку анализа дин

In [9]:
'''
Integrate all articles texts in one text
'''
all_article_texts = df['article_text']
integrated_text = '' # This variable integrates all 46957 articles texts in same one
SPACE = ' '
file_to_write = open(
  '/content/drive/MyDrive/Science/Datasets/cyberleninka-sociology-articles/cyberleninka-sociology-articles'
  + '__2-0_articles-content/integrated-dataframes/all-article-texts-in-one-text__actuality-25-02-2021.txt',
  'a'
)

for article_text in all_article_texts:
  try:
    textAsArr = transformFromStrToArr(article_text)
    restoredText = restoreOneArticleText(textAsArr)
    file_to_write.write(restoredText + SPACE)
    file_to_write.close()
  except:
    pass

integrated_text

''